In [1]:
# ECGR 4106
# HW 2
# Trevor Goad

In [2]:
import numpy as np 
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import time

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
housing = pd.DataFrame(pd.read_csv("Housing.csv")) 

housing.head() 

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [5]:
varlist =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
 
# Defining the map function 
def binary_map(x): 
    return x.map({'yes': 1, "no": 0}) 
 
# Applying the function to the housing list 
housing[varlist] = housing[varlist].apply(binary_map)

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import MinMaxScaler, StandardScaler

normalScaler = MinMaxScaler()
standardScaler = StandardScaler()

housing = housing.iloc[:, [0,1,2,3,4,10]]
housing = normalScaler.fit_transform(housing)

X = housing[:, [1,2,3,4,5]]
Y = housing[:, 0]

In [6]:
#Splitting the Data into Training and Testing Sets 
from sklearn.model_selection import train_test_split 
 
np.random.seed(0) 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 413)

In [7]:

t_u_train = X_train
t_u_test = X_test
t_c_train = Y_train
t_c_test = Y_test

t_u_train = torch.FloatTensor(t_u_train)
t_u_test = torch.FloatTensor(t_u_test)
t_c_train = torch.FloatTensor(t_c_train)
t_c_test = torch.FloatTensor(t_c_test)

t_un_train = t_u_train.to(device)
t_un_test = t_u_test.to(device)
t_c_train = t_c_train.to(device)
t_c_test = t_c_test.to(device)

In [8]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train,
                 t_u_val, t_c_train, t_c_val):
    start = time.time()
    for epoch in range(1, n_epochs+1):
        t_p_train = model(t_u_train)
        loss_train = loss_fn(t_p_train, t_c_train)
        
        t_p_val = model(t_u_val)
        
        loss_val = loss_fn(t_p_val, t_c_val)
    
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()
    
        if epoch == 1 or epoch % 10 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
              f" Validation loss {loss_val.item():.4f}")
    stop = time.time()
    print(f"Training time: {stop - start}s")
    return 0

In [9]:
seq_model_1 = nn.Sequential(
            nn.Linear(5, 8),
            nn.Tanh(),
            nn.Linear(8, 1))
seq_model_1.to(device)

Sequential(
  (0): Linear(in_features=5, out_features=8, bias=True)
  (1): Tanh()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

In [10]:
optimizer = optim.SGD(seq_model_1.parameters(), lr=1e-3)

training_loop(
    n_epochs = 200,
    optimizer = optimizer,
    model = seq_model_1,
    loss_fn = nn.MSELoss(),
    t_u_train = t_un_train,
    t_u_val = t_un_test,
    t_c_train = t_c_train,
    t_c_val = t_c_test)

print('done')



Epoch 1, Training loss 0.0265, Validation loss 0.0337
Epoch 10, Training loss 0.0265, Validation loss 0.0337
Epoch 20, Training loss 0.0265, Validation loss 0.0337
Epoch 30, Training loss 0.0265, Validation loss 0.0336
Epoch 40, Training loss 0.0264, Validation loss 0.0336
Epoch 50, Training loss 0.0264, Validation loss 0.0335
Epoch 60, Training loss 0.0264, Validation loss 0.0335
Epoch 70, Training loss 0.0264, Validation loss 0.0335
Epoch 80, Training loss 0.0263, Validation loss 0.0334
Epoch 90, Training loss 0.0263, Validation loss 0.0334
Epoch 100, Training loss 0.0263, Validation loss 0.0334
Epoch 110, Training loss 0.0263, Validation loss 0.0334
Epoch 120, Training loss 0.0263, Validation loss 0.0333
Epoch 130, Training loss 0.0263, Validation loss 0.0333
Epoch 140, Training loss 0.0262, Validation loss 0.0333
Epoch 150, Training loss 0.0262, Validation loss 0.0333
Epoch 160, Training loss 0.0262, Validation loss 0.0333
Epoch 170, Training loss 0.0262, Validation loss 0.0332
Epo

In [11]:
seq_model_2 = nn.Sequential(
            nn.Linear(5, 8),
            nn.Tanh(),
            nn.Linear(8, 4),
            nn.Tanh(),
            nn.Linear(4,2),
            nn.Tanh(),
            nn.Linear(2,1))
seq_model_2.to(device)

Sequential(
  (0): Linear(in_features=5, out_features=8, bias=True)
  (1): Tanh()
  (2): Linear(in_features=8, out_features=4, bias=True)
  (3): Tanh()
  (4): Linear(in_features=4, out_features=2, bias=True)
  (5): Tanh()
  (6): Linear(in_features=2, out_features=1, bias=True)
)

In [12]:
optimizer = optim.SGD(seq_model_1.parameters(), lr=1e-3)

training_loop(
    n_epochs = 200,
    optimizer = optimizer,
    model = seq_model_1,
    loss_fn = nn.MSELoss(),
    t_u_train = t_un_train,
    t_u_val = t_un_test,
    t_c_train = t_c_train,
    t_c_val = t_c_test)

print('done')



Epoch 1, Training loss 0.0262, Validation loss 0.0332
Epoch 10, Training loss 0.0262, Validation loss 0.0332
Epoch 20, Training loss 0.0262, Validation loss 0.0332
Epoch 30, Training loss 0.0261, Validation loss 0.0332
Epoch 40, Training loss 0.0261, Validation loss 0.0331
Epoch 50, Training loss 0.0261, Validation loss 0.0331
Epoch 60, Training loss 0.0261, Validation loss 0.0331
Epoch 70, Training loss 0.0261, Validation loss 0.0331
Epoch 80, Training loss 0.0261, Validation loss 0.0331
Epoch 90, Training loss 0.0261, Validation loss 0.0331
Epoch 100, Training loss 0.0261, Validation loss 0.0331
Epoch 110, Training loss 0.0261, Validation loss 0.0331
Epoch 120, Training loss 0.0261, Validation loss 0.0331
Epoch 130, Training loss 0.0261, Validation loss 0.0330
Epoch 140, Training loss 0.0261, Validation loss 0.0330
Epoch 150, Training loss 0.0261, Validation loss 0.0330
Epoch 160, Training loss 0.0261, Validation loss 0.0330
Epoch 170, Training loss 0.0261, Validation loss 0.0330
Epo

In [13]:
#problem 2

In [14]:
from torchvision import datasets
from torchvision import transforms
data_path = './tmp/tensorflow/mnist/input_data'
cifar10 = datasets.CIFAR10(data_path, train = True, download = True,
                          transform=transforms.ToTensor())
cifar10_val = datasets.CIFAR10(data_path, train = False, download = True,
                              transform=transforms.ToTensor())

Files already downloaded and verified
Files already downloaded and verified


In [15]:
imgs = torch.stack([img_t for img_t, _ in cifar10_val], dim = 3)
imgs.view(3, -1).mean(dim=1), imgs.view(3, -1).std(dim=1)

(tensor([0.4942, 0.4851, 0.4504]), tensor([0.2467, 0.2429, 0.2616]))

In [16]:
cifar10 = datasets.CIFAR10(data_path, train = True, download = False,
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.4914, 0.4822, 0.4465),
                                                   (0.2470, 0.2435, 0.2616))
                           ]))

cifar10_val = datasets.CIFAR10(data_path, train = False, download = False,
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.4942, 0.4851, 0.4504),
                                                   (0.2467, 0.2429, 0.2616))
                           ]))

In [17]:
from torch.utils.data import DataLoader
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                            shuffle=True)

model = nn.Sequential(
        nn.Linear(3072, 512),
        nn.Tanh(),
        nn.Linear(512, 10),
        nn.LogSoftmax(dim=1))
model.to(device)

learning_rate = 1e-2

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.NLLLoss()

n_epochs = 300

start = time.time()

for epoch in range(n_epochs+1):
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        batch_size = imgs.shape[0]
        outputs = model(imgs.view(batch_size, -1))
        loss = loss_fn(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch % 25 ==0):
        print("Epoch: %d, Loss: %f" % (epoch, float(loss)))
    
stop = time.time()
print(f"Training time: {stop - start}s")

Epoch: 0, Loss: 1.852000
Epoch: 25, Loss: 1.412853
Epoch: 50, Loss: 1.106567
Epoch: 75, Loss: 0.940826
Epoch: 100, Loss: 0.606194
Epoch: 125, Loss: 0.493893
Epoch: 150, Loss: 0.328648
Epoch: 175, Loss: 0.271958
Epoch: 200, Loss: 0.181335
Epoch: 225, Loss: 0.130539
Epoch: 250, Loss: 0.110038
Epoch: 275, Loss: 0.084745
Epoch: 300, Loss: 0.060938
Training time: 2974.2855870723724s


In [18]:
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                        shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        batch_size = imgs.shape[0]
        outputs = model(imgs.view(batch_size, -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
    
print("Accuracy: %f", correct/total)

Accuracy: %f 0.4695


In [ ]:
from torch.utils.data import DataLoader
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                            shuffle=True)

model = nn.Sequential(
        nn.Linear(3072, 2150),
        nn.Tanh(),
        nn.Linear(2150, 1120),
        nn.Tanh(),
        nn.Linear(1120, 512),
        nn.Tanh(),
        nn.Linear(512, 10),
        nn.LogSoftmax(dim=1))
model.to(device)

learning_rate = 1e-2

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.NLLLoss()

n_epochs = 300

start = time.time()

for epoch in range(n_epochs+1):
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        batch_size = imgs.shape[0]
        outputs = model(imgs.view(batch_size, -1))
        loss = loss_fn(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if (epoch % 25 ==0):
        print("Epoch: %d, Loss: %f" % (epoch, float(loss)))
    
stop = time.time()
print(f"Training time: {stop - start}s")

Epoch: 0, Loss: 2.063636
Epoch: 25, Loss: 0.920640
Epoch: 50, Loss: 0.019232


In [20]:
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                        shuffle=False)
correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        batch_size = imgs.shape[0]
        outputs = model(imgs.view(batch_size, -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
    
print("Accuracy: %f", correct/total)

Accuracy: %f 0.4806
